# Using Vector Geometry in Cal-Adapt API

In [ ]:
import requests 
import pprint

For getting climate data for an area, you can use the `g` query parameter to provide polygon geometry. Another way to provide geometry is to use polygons from the boundary datasets stored in Cal-Adapt API using the `ref` parameter. For a list of boundary datasets available through Cal-Adapt refer to the [documentation](https://berkeley-gif.github.io/caladapt-docs/data-catalog.html#vector-data).

In the cell below we are querying the `counties` resource to get a reference to the county polygon intersected by a point.

In [ ]:
# Name of vector resource in API.
resource = 'counties'
url = 'http://api.cal-adapt.org/api/%s/' % resource
# HTTP header
headers = {'ContentType': 'json'}

# Point location in geojson format
lat = 38.5938
lng = -121.4687
point = '{"type":"Point","coordinates":[%s,%s]}' % (lng, lat)

# Params
params = {'intersects': point, 'srs': 4326, 'simplify': .0001, 'precision': 4}

# Get county geometry
response = requests.get(url, params=params, headers=headers)
print(response.url)
print()
if response.ok:
    data = response.json()
    pprint.pprint(data)

Format a Uniform Resource Identifier (URI) for the polygon

In [ ]:
features = data['features']
polygon = features[0]
polygon_uri = '/api/%s/%s/' % (resource, polygon['id'])
print ('Uniform Resource Identifier for county:', polygon_uri)

Get climate data for polygon

In [ ]:
params = {
    'pagesize': 100,
    'ref': polygon_uri,
    'stat': 'mean'
}
url = 'http://api.cal-adapt.org/api/series/tasmax_year_CNRM-CM5_rcp45/rasters/'

# Make request
response = requests.get(url, params=params, headers=headers)

# It is a good idea to check there were no problems with the request.
if response.ok:
    data = response.json()
    # Get a list of Raster Stores from results property of data object
    results = data['results']
    print('Tiemseries for the this polygon:')
    # Iterate through the list and print the event and image property of each Raster Store
    for item in results:
        print('year:', item['event'], 'value:', item['image'], item['units'])

### Your turn

1. Get climate data for a different area. Change the `resource` type and/or change coordinates for `point` location